<h1> Tag Analysis </h1>

<hr>

Probably by looking at the abstract will be easier to tag a document.

In [50]:
import pickle
import pandas as pd
import datetime
from datetime import datetime

df = pd.read_csv('Data/processed_data_v3.csv', sep=';')
df['after_dec'] = df.apply(lambda row: check_date(row['date']), axis = 1)

def check_date(date):
    data = '2019-12-01'
    data = datetime.strptime(data, '%Y-%m-%d').date()
    if len(date.split('-')) < 3:
        date += "-01-01"
    date = datetime.strptime(date, '%Y-%m-%d').date()
    return date > data

In [2]:
# Load abstracts dictionary
with open('Data/ranking_dict/document_frequencies_abstract_proc_v3.p', 'rb') as fp:
    document_frequencies = pickle.load(fp)
with open('Data/ranking_dict/term_frequencies_abstract_proc_v3.p', 'rb') as fp:
    term_frequencies = pickle.load(fp)
with open('Data/ranking_dict/document_length_abstract_proc_v3.p', 'rb') as fp:
    document_length = pickle.load(fp)

In [3]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
other = ['et', 'al', 'also', 'fig', 'cell', 'virus', 'diseas', 'result', 'infect', 'studi', 'use']
remove = set()
for word in document_frequencies:
    if word in stopwords or word.isdigit() or word in other:
        remove.add(word)
for word in remove:
    del document_frequencies[word]

In [4]:
list_term = sorted(document_frequencies.items(), key=lambda x:x[1], reverse=True)

In [5]:
list_term[:10]

[('abstract', 8699),
 ('show', 8054),
 ('viral', 8012),
 ('develop', 7801),
 ('find', 7647),
 ('effect', 7342),
 ('human', 7314),
 ('high', 7264),
 ('sever', 7244),
 ('includ', 7217)]

In [6]:
n_doc = 0
for key in document_frequencies:
    if 'cov' in key and '19' in key:
        print(key)
        n_doc += document_frequencies[key]
    if 'sars' in key and 'cov' in key and '2' in key:
        print(key)
        n_doc += document_frequencies[key]
        
print(n_doc)

2019ncovr
covid19
ncov2019
sarscov2
diagnosis_covid19
cov19
ofcovid19
covod19
2019ncov
25


In [73]:
found = {'covid': (0,0), 'cov,19': (0,0), 'sar,cov,2': (0,0), 'wuhan': (0,0), 'cov,20': (0,0), 'cov,2': (0,0), 'cov,2,2': (0,0), 'respiratori': (0,0)}
for _, row in df.iterrows():
    abstract = row['abstract']
    date = row['date']
    if type(abstract) != str:
        continue
    elif 'covid' in abstract:
        a,b = found['covid']
        a += 1
        if check_date(date):
            b += 1
        found['covid'] = (a,b)
    elif 'cov' in abstract and '19 ' in abstract:
        a,b = found['cov,19']
        a += 1
        if check_date(date):
            b += 1
        found['cov,19'] = (a,b)
    elif 'cor' in abstract and '19 ' in abstract:
        a,b = found['cov,19']
        a += 1
        if check_date(date):
            b += 1
        found['cov,19'] = (a,b)
    elif 'sar' in abstract and 'cov' in abstract and '2 ' in abstract:
        a,b = found['sar,cov,2']
        a += 1
        if check_date(date):
            b += 1
        found['sar,cov,2'] = (a,b)
    elif 'wuhan' in abstract:
        a,b = found['wuhan']
        a += 1
        if check_date(date):
            b += 1
        found['wuhan'] = (a,b)
    elif 'cov' in abstract and '20 ' in abstract:
        a,b = found['cov,20']
        a += 1
        if check_date(date):
            b += 1
            #print(abstract + '\n')
        found['cov,20'] = (a,b)
    elif 'cov' in abstract and '2 ' in abstract:        
        a,b = found['cov,2,2']
        a += 1
        if check_date(date):
            b += 1
        found['cov,2'] = (a,b)
    elif 'respirator' in abstract:        
        a,b = found['respiratori']
        a += 1
        if check_date(date):
            b += 1
        found['respiratori'] = (a,b)

total = 0
total_true = 0
for key in found:
    a,b = found[key]
    total += a
    total_true += b
    

In [74]:
print(found)
print(total)
print(total_true)

{'covid': (7, 7), 'cov,19': (986, 697), 'sar,cov,2': (26, 1), 'wuhan': (1, 1), 'cov,20': (113, 10), 'cov,2': (1, 0), 'cov,2,2': (0, 0), 'respiratori': (5635, 311)}
6769
1027


In [60]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer
stemmer = SnowballStemmer("english")

word = 'respiratory'
word = stemmer.stem(WordNetLemmatizer().lemmatize(word.lower(), pos='v'))

print(word)

respiratori


In [53]:
df[df.after_dec == True].abstract.isna().sum()

197

In [54]:
len(df[df.after_dec == True])

2507